In [1]:
!pip install librosa

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/N/soft/rhel7/deeplearning/Python-3.9.6/bin/python3.9 -m pip install --upgrade pip' command.


In [2]:
import torch
from torchaudio.datasets import SPEECHCOMMANDS
import os
import torchaudio
from os.path import join, exists
import matplotlib.pyplot as plt
from IPython.display import Audio
import librosa
from math import log2
import numpy as np
from tqdm import tqdm
from multiprocess import Pool
import functools
import shutil

## Load Data (Google Speech Commands V2)

In [3]:
# modified version of 
#.  https://pytorch.org/tutorials/intermediate/speech_command_classification_with_torchaudio_tutorial.html

class SubsetSC(SPEECHCOMMANDS):
    def __init__(self, subset: str = None):
        super().__init__("../data/", download=True)

        def load_list(filename):
            filepath = os.path.join(self._path, filename)
            with open(filepath) as fileobj:
                return [os.path.join(self._path, line.strip()) for line in fileobj]

        if subset == "validation":
            self._walker = load_list("validation_list.txt")
        elif subset == "testing":
            self._walker = load_list("testing_list.txt")
        elif subset == "training":
            excludes = load_list("validation_list.txt") + load_list("testing_list.txt")
            excludes = set(map(os.path.abspath, excludes))
            self._walker = [w for w in self._walker if os.path.abspath(w) not in excludes]
    
    def __getitem__(self, idx):
      fname = self._walker[idx]
      x = torchaudio.load(fname)[0]
      split = fname.split("/")
      word = split[-2]
      id = split[-1].rstrip(".wav")
      return (x, word, id)

In [4]:
train = SubsetSC("training")
val = SubsetSC("validation")
test = SubsetSC("testing")
print(len(train), len(val), len(test))

84843 9981 11005


In [5]:
idx = 0
word_index = {}
for dat in train:
  word = dat[1]
  if word not in word_index:
    word_index[word] = idx
    idx += 1
print("Num labels:", len(word_index.keys()))
print(word_index.keys())

Num labels: 35
dict_keys(['backward', 'bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'follow', 'forward', 'four', 'go', 'happy', 'house', 'learn', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'visual', 'wow', 'yes', 'zero'])


### Process Data

In [6]:
fmin = 100
fmax = 6000
bins = 50
hop_length = 64
bins_per_octave = int(bins / log2(fmax/fmin) + 0.5)
print(f"Creating constant-q filter with {bins_per_octave} bins per octave")

Creating constant-q filter with 8 bins per octave


In [7]:
def normalize(X):
  # min-max
  d = X.max() - X.min()
  eps = 1e-4
  return (X - X.min()) / (d + eps)

In [8]:
def transform(data):
  x, word, id = data
  x = x.flatten().numpy()
  X = librosa.cqt(x, sr=16000, hop_length=hop_length, fmin=fmin, n_bins=bins, bins_per_octave=bins_per_octave)
  X = torch.tensor(X[np.newaxis]).abs()
  X = normalize(X)
  return x, X, word, id

In [9]:
idx = 8000
x = train[idx]
Audio(x[0][0].numpy(), rate=16000)

In [10]:
transformed = transform(x)
X = transformed[1][0]
word = transformed[2]
plt.title(word)
plt.imshow(X, aspect="auto")
print(word_index[word])

4


In [11]:
def transform_and_save(dirname, dat):
  x, word, id = data
  path = join(dirname, word, id + ".pt")
  if not exists(path): 
      x, X, word, id = transform(dat)
      label = word_index[word]
      item = (X, label)

      torch.save(item, path)

In [12]:
def save_data(dataset, dirname, num_workers=4):
  #shutil.rmtree(dirname, ignore_errors=True)
  for word in word_index.keys():
    word_dir = join(dirname, word)
    os.makedirs(word_dir, exist_ok=True)

  pool = Pool(num_workers)
  func = functools.partial(transform_and_save, dirname)
  t = tqdm(
    pool.imap_unordered(func, dataset),
    total=len(dataset),
    leave=False
  )
  count = 0
  for _ in t:
    count += 1

In [14]:
dataset_dir = "../data/SpeechCommands/processed"
save_data(train, join(dataset_dir, "train"))
save_data(val, join(dataset_dir, "val"))
save_data(test, join(dataset_dir, "test"))

 14%|█████▎                               | 1590/11005 [01:05<06:25, 24.40it/s]/N/u/skarukas/Carbonate/.local/lib/python3.9/site-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=128 is too small for input signal of length=107
  warnings.warn(


NameError: name 'requests' is not defined

In [17]:
path = join(dataset_dir, "word_indices.txt")
with open(path, "w") as f:
  f.writelines(" ".join(map(str, pair)) + "\n" for pair in word_index.items())